In [2]:
import time
import requests
from bs4 import BeautifulSoup
from selenium import webdriver

from selenium.webdriver import Chrome

# 뉴스 검색

In [4]:
def news_search(query='"삼성전자"+-주가+-배구+-농구', page=1, maxpage=2, s_day='', e_day=''):
#     query = ''
    in_page = 1
    in_maxpage = 2
   
    s_day = '2020.02.01'
    e_day = '2020.02.02'
    maxpage_t = (int(in_maxpage)-1)*10+1
    start_day = s_day.replace('.', '')
    end_day = e_day.replace('.', '')
#     link_list = []
#     title_list = []
    time_s = 0.5
    news_info_list = {}
    while in_page <= maxpage_t:
        link = 'https://search.naver.com/search.naver?&where=news&query='+query+'&sm=tab_pge&sort=0&photo=0&field=0&reporter_article=&pd=3&ds='+start_day+'&de='+end_day+'&docid=&nso=so:r,p:from20190107to20190131,a:all&mynews=0&cluster_rank=41&start='+str(page)+'&refresh_start=0'
        print(link)
        time.sleep(time_s)
        res = requests.get(link)
        html = res.text
        
        soup = BeautifulSoup(html, 'html.parser')
        main_list = soup.findAll('ul',{'class':'type01'})[0].findAll('li')
        
        info_list = {}
        end_news = len(main_list)
        for i in main_list:
            if [] != i.findAll('a', {"class":"_sp_each_url"}) and i.findAll('a', {"class":"_sp_each_title"}) != [] and 'https://news.naver.com/' in i.findAll('a', {"class":"_sp_each_url"})[0]['href']:
                info_list[i.findAll('a', {"class":"_sp_each_url"})[0]['href']]= {'title':i.findAll('a', {"class":"_sp_each_title"})[0].text, 
                                  'news_comp':i.findAll('span', {'class':'_sp_each_source'})[0].text, 
                                  'link':i.findAll('a', {"class":"_sp_each_url"})[0]['href']}
        news_info_list.update(info_list)
        in_page += 10
    return soup, news_info_list

In [5]:
test, news_info_list = news_search()
# test

https://search.naver.com/search.naver?&where=news&query="삼성전자"+-주가+-배구+-농구&sm=tab_pge&sort=0&photo=0&field=0&reporter_article=&pd=3&ds=20200201&de=20200202&docid=&nso=so:r,p:from20190107to20190131,a:all&mynews=0&cluster_rank=41&start=1&refresh_start=0
https://search.naver.com/search.naver?&where=news&query="삼성전자"+-주가+-배구+-농구&sm=tab_pge&sort=0&photo=0&field=0&reporter_article=&pd=3&ds=20200201&de=20200202&docid=&nso=so:r,p:from20190107to20190131,a:all&mynews=0&cluster_rank=41&start=1&refresh_start=0


In [6]:
# max_news_ = test.findAll('div', {'class':'title_desc all_my'})[0].text.split('/')[1]
# int(int(re.findall(r'[0-9]+', max_news_)[0])/9)+1

In [7]:
news_info_list

{'https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=101&oid=001&aid=0011371549': {'link': 'https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=101&oid=001&aid=0011371549',
  'news_comp': '연합뉴스',
  'title': '"크고 비싼 게 잘 팔린다"…삼성, 국내 프리미엄TV 시장 견인'},
 'https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=101&oid=029&aid=0002584859': {'link': 'https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=101&oid=029&aid=0002584859',
  'news_comp': '디지털타임스',
  'title': '삼성·SK·LG, 변화 감지… `분사 - 사명변경` 움직임'},
 'https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=101&oid=277&aid=0004616476': {'link': 'https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=101&oid=277&aid=0004616476',
  'news_comp': '아시아경제',
  'title': '삼성·LG, 생활가전 실적 호조 ‘프리미엄 제품 전략’으로 이어간다'},
 'https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=101&oid=437&aid=0000230163': {'link': 'https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=101&oid=437&aid=0000230163',
  'news_comp': 'JTBC언론사 

# 뉴스 데이터 추출

In [8]:
addr = "https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=103&oid=366&aid=0000448680"
addr ='https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=101&oid=011&aid=0003569092'
def make_det(addr):
    chrome_ = 'D:\chromedriver_win32\chromedriver.exe'
    driver = webdriver.Chrome(chrome_)
    # driver = Chrome()
    driver.get(addr)
    date = driver.find_element_by_class_name('t11').text[:10]
    good_and_bad = driver.find_element_by_class_name('end_btn').text
    title = driver.find_element_by_class_name('tts_head').text
    content = driver.find_element_by_class_name('_article_body_contents').text
    time.sleep(0.5)
    comp = driver.find_element_by_class_name('changecmt_p').text.replace('이 기사의 댓글 정책은 ', '').replace('가 결정합니다.', '').replace('안내','')
    try:
        time.sleep(1)
        driver.find_element_by_xpath('//*[@id="cbox_module"]/div[2]/div[9]/a').click()
        while True:
            try:
                time.sleep(1)
                driver.find_element_by_xpath('//*[@id="cbox_module"]/div[2]/div[9]/a/span').click()
            except:
                break
            reviews_number = int(driver.find_element_by_class_name('u_cbox_count').text)
            reviews = [driver.find_element_by_xpath('//*[@id="cbox_module"]/div[2]/div[8]/ul/li['+str(j)+']/div[1]/div/div[3]/span[1]').text[:10] for j in range(1, reviews_number+1)]
            init_date = {}
            for k in reviews:
                init_date[k] = 0
            for k in reviews:
                init_date[k] += 1
    except:
        pass
    driver.close()
    try:
        output = {'title':title, 'review_count':init_date, 'date':date, 'content':content, 'news_comp':comp, 'good_bad':good_and_bad, 'link':addr}
    except:
        output = {'title':title, 'review_count':None, 'date':date, 'content':content, 'news_comp':comp, 'good_bad':good_and_bad, 'link':addr}
    return output

In [9]:
count = 0
for i in news_info_list:
    if count == 1:
        break
    count+=1
    
    print(i)
#     print()
    new_dict = {}
    new_dict[i] = make_det(i)
    new_dict[i].update(news_info_list[i])
    print(new_dict)

https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=105&oid=015&aid=0004282558
{'https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=105&oid=015&aid=0004282558': {'title': "삼성폰 꺾고 1위 탈환한 애플, '신종 코로나'에 기세 꺾이나", 'news_comp': '한국경제언론사 선정', 'link': 'https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=105&oid=015&aid=0004282558', 'good_bad': '좋아요\n149\n훈훈해요\n4\n슬퍼요\n10\n화나요\n16\n후속기사 원해요\n19\n이 기사를 추천합니다 28\n안내\n보내기', 'date': '2020.02.02', 'content': '애플 전체 매출 20%가량 차지하는 애플 최대 시장인 中\n애플, 오는 9일까지 중국 모든 매장 임시 폐장\n중국 위치한 대만 폭스콘 등 생산 체인에 악영향 가능성까지\n신종 코로나 우려에 애플 CEO, 직접 \'차이나리스크\' 거론\n사진=연합뉴스(로이터)\n\n지난해 4분기 스마트폰 판매량 1위 자리에 올라선 애플이 올해 연초부터 신종 코로나 바이러스 감염증(우한 폐렴) 확산이라는 뜻밖의 암초를 만났다. 애플이 중국 본토의 모든 매장을 임시 폐쇄한다고 밝히며 최대 시장인 중국의 스마트폰 수요가 줄어드는 데다, 중국 현지에 핵심 생산 체인이 모두 밀집돼 있어 아이폰 생산까지도 차질을 빚을 것이라는 전망이 나온다.\n\n시장조사업체 카운터포인트는 지난해 4분기 애플의 스마트폰 판매량이 7290만대, 전체 시장 점유율을 18%를 기록해 7000만대를 판매한 2위 삼성전자를 근소하게 앞질렀다고 지난 31일(현지시간) 밝혔다. 분기 기준 애플이 삼성을 꺾은 건 2년 만이다. 지난해 9월 출시한 \'아이폰 11\' 시리즈의 흥행이 컸다.\

In [7]:
output

{'content': "中 부품사 입찰 전면개방 이어\n국내시장도 수익성 방어 주문\n내달 출시 소형SUV 셀토스 등\n한국·금호타이어 적용모델 늘릴듯\n\n[서울경제] 정의선 현대자동차그룹 총괄수석부회장이 “국내 시장에서도 원가경쟁력을 높이라”고 주문한 것으로 파악됐다. 올해 고고도미사일방어체계(THAAD·사드) 사태 이후 판매실적 회복이 더딘 중국 법인에 현지 부품사들의 입찰을 전면 개방한 데 이어 내수시장도 수익성을 방어하라는 의미다. 원가 경쟁력 개선 주문으로 국내 자동차부품 시장에서 국산 타이어 채용이 확대될 조짐에 따라 타이어업계의 기대감도 커지고 있다.\n\n12일 자동차업계의 한 고위관계자는 “정 수석부회장이 내부에 원가경쟁력과 수익성을 개선하라는 방침을 내렸다”며 “특히 소형 스포츠유틸리티차량(SUV) 등의 상품성도 높여 제품 경쟁력과 이익률 모두 방어하라는 의미”라고 설명했다.\n\n현대·기아자동차는 최근 몇 년간 미국 시장에서 SUV 라인업 부족으로 부진을 겪은 데 이어 중국 시장에서 사드 사태를 맞으며 판매량이 179만대에서 116만대까지 하락한 상황이다. 지난 2011년 10%를 넘던 영업이익률은 지난해 2%대까지 추락했다. 이에 지난해부터 주주들이 수익성을 개선하라는 요구를 공개적으로 했다. 최병철 현대차 부사장은 올 4월 열린 콘퍼런스콜에서 “원가 절감을 통해 올해 4% 이상 영업이익률을 기록할 수 있도록 집중하겠다”고 발표했다. 이와 함께 현대·기아차는 “중국 법인의 경우 현지 부품업체들의 입찰을 전면 개방하겠다”고 밝혔다. 꼭 글로벌 부품기업이나 한국 업체가 아니라도 엄격한 품질 기준을 맞추면 중국 부품도 채용해 이익률을 높이겠다는 것이다. 한 완성차 업체 관계자는 “미중 무역전쟁으로 주요 시장과 신흥국이 불안해지면서 이익률을 회복하지 않고는 장기전에 돌입하기 힘든 상황이 됐다”며 “주주들의 요구가 아니라도 2%대의 이익률로는 살아남기 힘들다”고 말했다.\n\n현대·기아차는 미국 시장에서는 팰리세이드와 텔루라이드 등 대형 SUV,